In [2]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2

In [51]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch import Tensor
from torch.nn.utils.rnn import pack_sequence, pad_sequence, pack_padded_sequence, pad_packed_sequence, PackedSequence

In [52]:
import datetime
from typing import Final, Type, Union

import numpy as np
from numpy import datetime64, ndarray, timedelta64
from numpy.typing import ArrayLike, NDArray
import pandas
from pandas import Timedelta, Timestamp

TimeStampLike = Union[str, datetime.datetime, np.datetime64, pandas.Timestamp]
TimeDeltaLike = Union[str, datetime.timedelta, np.timedelta64, pandas.Timedelta]

In [53]:
a, b = False, True

In [49]:
Timedelta("900s")

In [56]:
rng = np.random.default_rng()

rng.choice(np.arange(10), size=None)

In [43]:
np.arange(15)[a:-b or None]

In [13]:
freq = Timedelta("15m")

In [15]:
Timestamp("today")

In [16]:
(Timestamp("today")+Timedelta("24h")).round(freq)

In [3]:
import tsdm

In [6]:
tsdm.random.timesamples(num=10, freq="15s")

In [2]:
class VaryingLengthDataset(Dataset):
    def __init__(self, n, dmin, dmax):
        super().__init__()
        self.size = n
        self.data = [torch.randint(0, 10, (np.random.randint(dmin, dmax), 2)) for _ in range(n)]
        
    # a commemt
    def __len__(self):
        return self.size
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __repr__(self):
        return "".join(repr(x.shape) for x in self.data)
    
def myfun():
    pass

In [137]:
import tsdm

In [ ]:
tsdm.

In [12]:
pandas.Timestamp("2:00")

In [129]:
units = { u:np.timedelta64(1, u) for u in ("Y", "M", "W", "D", "h", "m", "s", "us", "ns", "ps", "fs", "as")}
base_unit = next(u for u, val in units.items() if freq >= val)

In [135]:
freq = pandas.Timedelta("15m")

In [136]:


base

In [90]:
now = pandas.Timestamp("now")
tom = pandas.Timestamp("now") + pandas.Timedelta("24h")


In [91]:
date_range = pandas.date_range(now, tom, freq=Timedelta("17m"))

choice = np.random.default_rng().choice(date_range, size=10, replace=False)

In [96]:
choice

In [103]:
np.datetime64(15, 'm')

In [105]:
choice.astype(np.datetime64(15, 'm'))

In [74]:
np.diff(np.sort(choice))

In [82]:
import inspect

In [83]:
inspect.getsource(Timestamp)

In [7]:
import numpy

In [2]:
import pandas

In [6]:
type(pandas.Timedelta("1s"))

In [116]:
datetime.datetime?

In [100]:
import datetime

In [112]:
type(datetime.timedelta(seconds=3))

In [113]:
issubclass(np.timedelta64, datetime.timedelta)

In [108]:
np.array(datetime.timedelta(seconds=5), dtype="timedelta64[s]").dtype

In [98]:
dir(np.typing)

In [20]:
import xarray


In [21]:
from tsdm.datasets import InSilicoData


In [25]:
dir(InSilicoData)

In [75]:
dsets = []

for key, val in InSilicoData.dataset.items():
    ds = df.to_xarray()
    ds = ds.assign_coords(run_id = key)
    dsets.append(ds)

In [77]:
dsets

In [96]:
np.datetime64?

In [91]:
import numpy as np
np.datetime64('now')

In [95]:
np.datetime64(24, 'h')

In [84]:
xarray.concat(dsets, dim="run_id").to_dataframe()

In [38]:
arrays = [xarray.DataArray(df, name=key) for key, df in InSilicoData.dataset.items()  ]

In [67]:
arrays[0]

In [40]:
xarray.merge( arrays)

In [50]:
df = list(InSilicoData.dataset.values())[0]
df.name = "agagaga"

In [26]:
xarray.Dataset.from_dict(InSilicoData.dataset)

In [19]:
dir(VaryingLengthDataset)

### Batching this Dataset won't work:

In [3]:
MyDataset = VaryingLengthDataset(100, 3, 7)

try:
    next(iter(DataLoader(MyDataset, batch_size=10)))
    print(x)
except Exception as E:
    print(E)

### Let's try with a custom collate_fn

In [4]:
np.random.randint(0, len(MyDataset), 20)

In [5]:
batch = [MyDataset[idx] for idx in np.random.randint(0, len(MyDataset), 5)]
batch

In [6]:
pad_sequence(batch, batch_first=True).shape

In [7]:
def collate_list(batch):
    return batch

try:
    x = next(iter(DataLoader(MyDataset, batch_size=5, collate_fn=collate_list)))
    print(x)
except Exception as E:
    print(E)

In [8]:
def collate_packed(batch):
    batch.sort(key=torch.Tensor.__len__, reverse=True)
    return pack_sequence(batch)

try:
    x = next(iter(DataLoader(MyDataset, batch_size=5, collate_fn=collate_packed)))
    print(x)
except Exception as E:
    print(E)

In [9]:
batch = next(iter(DataLoader(MyDataset, batch_size=5, collate_fn=collate_list)))
[x.T for x in batch]

In [10]:
batch_sorted = batch.copy()
batch_sorted.sort(key=torch.Tensor.__len__, reverse=True)
[x.T for x in batch_sorted]

In [11]:
batch_packed = pack_sequence(batch_sorted)
batch_packed.data.T, batch_packed.batch_sizes

In [12]:
batch_pad_packed, lengths = pad_packed_sequence(batch_packed, batch_first=True)
torch.swapaxes(batch_pad_packed, 1, 2), lengths

In [13]:
unpacked_batch = [x[:l].T for x, l in zip(batch_pad_packed, lengths)]
unpacked_batch

In [14]:
def reconstruct_batch(packed: PackedSequence) -> list[Tensor]:
    d = packed.data.shape[-1]
    dtype = packed.data.dtype
    device = packed.data.device
    
    lengths = []
    b0 = 0
    for b in batch_sizes:
        
        lengths += [b]

In [15]:
reconstructed_batch = batch_pad_packed
reconstructed_batch

In [16]:
batch_pad_packed.shape

In [17]:
def unpack(packed):
    pass

In [18]:
a = MyDataset[:5]
a.sort(key=lambda x: len(x))
a